In [2]:
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

In [3]:
import shutil

import pandas as pd
import numpy as np

import tensorflow as tf
import tensorflow_transform as tft

# Training and Evaluation Data
Training and evaluation data should be provided in files already.

#### Check if train and eval files exists in /tmp/prod/
If not, please go back to ```Processing_ATL_JUNE.ipynb```

In [10]:
a_training_file = !ls /tmp/prod/atl_june_train-00001-of-*
a_training_file = a_training_file[0]
!wc -l $a_training_file

46327 /tmp/prod/atl_june_train-00001-of-00005


In [11]:
an_eval_file = !ls /tmp/prod/atl_june_eval-00000-of-*
an_eval_file = an_eval_file[0]
!wc -l $an_eval_file

15064 /tmp/prod/atl_june_eval-00000-of-00001


---
#### Have a look into the training data file

This data is at the **training data** stage. It's got all and only the columns we want. Is has been normalized and integerized. We'll use ```tf.feature_column``` to further process categorical features.

In [12]:
ALL_COLUMNS = [
    'YEAR', 'MONTH', 'DEP_DOW', 'AIRLINE', 
    'DEP_T', 'DEP', 'DEP_LAT', 'DEP_LON', 'DEP_DELAY',
    'MEAN_TEMP_DEP', 'MEAN_VIS_DEP', 'WND_SPD_DEP', 
    'ARR_DELAY', 'ARR', 'ARR_LAT', 'ARR_LON', 
    'MEAN_TEMP_ARR', 'MEAN_VIS_ARR', 'WND_SPD_ARR', 'DIFF_LAT', 'DIFF_LON', 'DISTANCE']

In [13]:
probe = pd.read_csv(a_training_file, names=ALL_COLUMNS)

In [14]:
probe.sample(frac=1.0)[:10]

,YEAR,MONTH,DEP_DOW,AIRLINE,DEP_T,DEP,DEP_LAT,DEP_LON,DEP_DELAY,MEAN_TEMP_DEP,...,ARR_DELAY,ARR,ARR_LAT,ARR_LON,MEAN_TEMP_ARR,MEAN_VIS_ARR,WND_SPD_ARR,DIFF_LAT,DIFF_LON,DISTANCE
40459,2002,6,3,Delta Air Lines Inc.: DL,650,ATL,33.63,-84.42,0.056492,0.865546,...,2.0,IAD,38.94,-77.46,0.518053,0.009206,0.005201,0.488613,0.864046,0.102870
38489,2007,6,1,Delta Air Lines Inc.: DL,2010,ATL,33.63,-84.42,0.114965,0.478992,...,60.0,SAV,32.12,-81.20,0.516483,0.005504,0.013401,0.331723,0.823883,0.030547
23588,2010,6,1,Delta Air Lines Inc.: DL,1636,ATL,33.63,-84.42,0.113974,0.920168,...,52.0,PHX,33.43,-112.01,0.648352,0.009407,0.005301,0.361859,0.493020,0.340833
15759,2011,6,7,Delta Air Lines Inc.: DL,755,ATL,33.63,-84.42,0.066402,0.747899,...,3.0,MCO,28.42,-81.30,0.667190,0.009106,0.006001,0.246607,0.822809,0.073693
24660,2003,6,7,Delta Air Lines Inc.: DL,805,ATL,33.63,-84.42,0.054509,0.352941,...,-6.0,PIT,40.49,-80.23,0.450549,0.009307,0.003400,0.524270,0.834300,0.101445
24585,2003,6,7,Delta Air Lines Inc.: DL,2000,ATL,33.63,-84.42,0.054509,0.289916,...,12.0,PIT,40.49,-80.23,0.312402,0.009106,0.008701,0.524270,0.834300,0.101445
34953,2004,6,6,United Air Lines Inc.: UA,750,ATL,33.63,-84.42,0.051536,0.726891,...,-8.0,SFO,37.61,-122.37,0.389325,0.009407,0.011201,0.458017,0.381765,0.465416
32453,2011,6,1,AirTran Airways Corporation: FL,1514,ATL,33.63,-84.42,0.167493,0.882353,...,106.0,SAT,29.53,-98.46,0.706436,0.009407,0.005701,0.272142,0.638531,0.179699
21730,2004,6,3,America West Airlines Inc.: HP (Merged with US...,1730,ATL,33.63,-84.42,0.054509,0.214286,...,0.0,PHX,33.43,-112.01,0.830455,0.009407,0.005000,0.361859,0.493020,0.340833
17793,2005,6,2,AirTran Airways Corporation: FL,1030,ATL,33.63,-84.42,0.050545,0.571428,...,-6.0,PHL,39.87,-75.24,0.618524,0.007005,0.006101,0.510007,0.887887,0.132846


In [15]:
probe.describe()

,YEAR,MONTH,DEP_DOW,DEP_T,DEP_LAT,DEP_LON,DEP_DELAY,MEAN_TEMP_DEP,MEAN_VIS_DEP,WND_SPD_DEP,ARR_DELAY,ARR_LAT,ARR_LON,MEAN_TEMP_ARR,MEAN_VIS_ARR,WND_SPD_ARR,DIFF_LAT,DIFF_LON,DISTANCE
count,46327.000000,46327.0,46327.000000,46327.000000,46327.00,46327.00,46327.000000,46327.000000,46327.000000,46327.000000,46327.000000,46327.000000,46327.000000,46327.000000,46327.000000,46327.000000,46327.000000,46327.000000,46327.000000
mean,2006.769422,6.0,3.962722,1461.020398,33.63,-84.42,0.068088,0.623244,0.848384,0.357424,14.837050,35.243593,-89.125058,0.584110,0.008647,0.006283,0.403579,0.738777,0.152751
std,2.752466,0.0,1.972015,471.069290,0.00,0.00,0.035096,0.184477,0.182975,0.142504,38.861123,4.820357,14.825564,0.140490,0.001264,0.002754,0.110889,0.159209,0.127843
min,2002.000000,6.0,1.000000,544.000000,33.63,-84.42,0.007929,0.000000,0.000000,0.000000,-50.000000,28.420000,-122.590000,0.155416,0.000000,0.000200,0.246607,0.379403,0.015283
25%,2004.000000,6.0,2.000000,1030.000000,33.63,-84.42,0.051536,0.491596,0.765625,0.255814,-6.000000,29.990000,-90.500000,0.486656,0.008306,0.004500,0.282724,0.724012,0.073693
50%,2007.000000,6.0,4.000000,1455.000000,33.63,-84.42,0.054509,0.626050,0.921875,0.348837,3.000000,35.040000,-81.300000,0.610675,0.009106,0.005801,0.398896,0.822809,0.102870
75%,2009.000000,6.0,6.000000,1854.000000,33.63,-84.42,0.068385,0.764706,0.984375,0.441860,20.000000,39.870000,-77.460000,0.675039,0.009407,0.007501,0.510007,0.864046,0.159431
max,2011.000000,6.0,7.000000,2340.000000,33.63,-84.42,0.634291,1.000000,1.000000,1.000000,593.000000,45.580000,-70.300000,1.000000,0.018913,0.024102,0.641362,0.940936,0.473032


---
# ```tf.data``` Input Functions

In [16]:
print(list(probe.keys()))

['YEAR', 'MONTH', 'DEP_DOW', 'AIRLINE', 'DEP_T', 'DEP', 'DEP_LAT', 'DEP_LON', 'DEP_DELAY', 'MEAN_TEMP_DEP', 'MEAN_VIS_DEP', 'WND_SPD_DEP', 'ARR_DELAY', 'ARR', 'ARR_LAT', 'ARR_LON', 'MEAN_TEMP_ARR', 'MEAN_VIS_ARR', 'WND_SPD_ARR', 'DIFF_LAT', 'DIFF_LON', 'DISTANCE']


---
For explorative purposes we allow ourselves to select a subset of the candidate features.

In [17]:
DEFAULTS = [[0], [0], [0], ["none"], 
            [0], ["none"], [0.], [0.], 
            [0.], [0.], [0.], 
            [0.], [0.], ["none"], [0.], [0.], 
            [0.], [0.], [0.], [0.], [0.], [0.]]

SELECT = [ALL_COLUMNS.index(col) for col in [
    'DEP_DOW', 'AIRLINE', 
    'DEP_T', 'DEP', 'DEP_LAT', 'DEP_LON', 'DEP_DELAY',
    'MEAN_TEMP_DEP', 'MEAN_VIS_DEP', 'WND_SPD_DEP', 
    'ARR_DELAY', 'ARR', 'ARR_LAT', 'ARR_LON', 
    'MEAN_TEMP_ARR', 'MEAN_VIS_ARR', 'WND_SPD_ARR', 'DIFF_LAT', 'DIFF_LON', 'DISTANCE'    
]]
SELECT

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

#### This reads from a CSV of pre-processed data

In [19]:
def make_input_fn(filename, mode, batch_size):
    def _input_fn():
        dataset = tf.data.TextLineDataset(filename)

        def decode_csv(row):
            cols = tf.decode_csv(row, select_cols=SELECT, record_defaults=[DEFAULTS[i] for i in SELECT])
            features = dict(zip([ALL_COLUMNS[i] for i in SELECT], cols))
            return features

        def calc_hod(features):
            features['DEP_T'] = features['DEP_T'] // 100
            return features

        def pop_target(features):
            target = features.pop('ARR_DELAY')
            return features, target
        
        if mode == tf.estimator.ModeKeys.TRAIN:
            dataset = dataset.shuffle(buffer_size=80000)
                
        dataset = (dataset.repeat()
                   .map(decode_csv)
                   .map(calc_hod)
                   .map(pop_target)
                   .batch(batch_size))

        return dataset.make_one_shot_iterator().get_next()
    
    return _input_fn

---
#### Verify that the input_functions return features and labels

In [20]:
train_input_fn = make_input_fn(
    a_training_file, mode=tf.estimator.ModeKeys.TRAIN, batch_size=2)

eval_input_fn = make_input_fn(
    an_eval_file, mode=tf.estimator.ModeKeys.EVAL, batch_size=2)  

In [21]:
try:
    sess.close()
except:
    pass
sess = tf.InteractiveSession()

In [22]:
input = train_input_fn()

In [23]:
sess.run(tf.global_variables_initializer())
sess.run(input)

({'AIRLINE': array(['Delta Air Lines Inc.: DL', 'ExpressJet Airlines Inc.: EV'],
        dtype=object),
  'ARR': array(['MSY', 'MEM'], dtype=object),
  'ARR_LAT': array([29.99, 35.04], dtype=float32),
  'ARR_LON': array([-90.25, -89.97], dtype=float32),
  'DEP': array(['ATL', 'ATL'], dtype=object),
  'DEP_DELAY': array([0.05450942, 0.06342914], dtype=float32),
  'DEP_DOW': array([4, 2], dtype=int32),
  'DEP_LAT': array([33.63, 33.63], dtype=float32),
  'DEP_LON': array([-84.42, -84.42], dtype=float32),
  'DEP_T': array([ 9, 17], dtype=int32),
  'DIFF_LAT': array([0.2827237, 0.3988958], dtype=float32),
  'DIFF_LON': array([0.7266967, 0.7297036], dtype=float32),
  'DISTANCE': array([0.07824197, 0.05709527], dtype=float32),
  'MEAN_TEMP_ARR': array([0.60439557, 0.67189956], dtype=float32),
  'MEAN_TEMP_DEP': array([0.59243685, 0.8865544 ], dtype=float32),
  'MEAN_VIS_ARR': array([0.00850595, 0.00930651], dtype=float32),
  'MEAN_VIS_DEP': array([0.875     , 0.74999994], dtype=float32),
  '

---
# Feature engineering

#### Bucketize lat and lon 

In [24]:
import google.datalab.bigquery as bq
query="""
select 
    distinct arrival_airport as airport, arrival_lat as lat, arrival_lon as lon 
from 
    `bigquery-samples.airline_ontime_data.flights`
"""
locations = bq.Query(query).execute().result().to_dataframe()
locations.describe()

,lat,lon
count,344.000000,344.000000
mean,38.491570,-98.531599
std,8.547964,21.746974
min,13.480000,-176.640000
25%,33.450000,-111.675000
50%,38.715000,-93.300000
75%,42.907500,-82.497500
max,71.280000,-64.800000


In [25]:
lat_boundaries = np.arange(10,80,5).tolist()
lat_boundaries

[10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75]

In [26]:
lon_boundaries = np.arange(-100, -55, 5).tolist()
lon_boundaries

[-100, -95, -90, -85, -80, -75, -70, -65, -60]

#### Straight features

In [27]:
dep_delay = tf.feature_column.numeric_column('DEP_DELAY')
mean_temp_dep = tf.feature_column.numeric_column('MEAN_TEMP_DEP')
mean_vis_dep = tf.feature_column.numeric_column('MEAN_VIS_DEP')
wnd_spd_dep = tf.feature_column.numeric_column('WND_SPD_DEP')
mean_temp_arr = tf.feature_column.numeric_column('MEAN_TEMP_ARR')
mean_vis_arr = tf.feature_column.numeric_column('MEAN_VIS_ARR')
wnd_spd_arr = tf.feature_column.numeric_column('WND_SPD_ARR')
diff_lat = tf.feature_column.numeric_column('DIFF_LAT')
diff_lon = tf.feature_column.numeric_column('DIFF_LON')
distance = tf.feature_column.numeric_column('DISTANCE')

#### Crossed and embedded: Arrival and departure coordinates and hour of week

In [28]:
cross_size = len(lat_boundaries) * len(lon_boundaries)

arr_lat = tf.feature_column.numeric_column('ARR_LAT')
arr_lat_b = tf.feature_column.bucketized_column(arr_lat, lat_boundaries)
arr_lon = tf.feature_column.numeric_column('ARR_LON')
arr_lon_b = tf.feature_column.bucketized_column(arr_lon, lon_boundaries)
arr_geo_cross = tf.feature_column.crossed_column(['ARR_LAT', 'ARR_LON'], cross_size)
arr_geo_emb = tf.feature_column.embedding_column(arr_geo_cross, 10)

dep_lat = tf.feature_column.numeric_column("DEP_LAT")
dep_lat_b = tf.feature_column.bucketized_column(dep_lat, lat_boundaries)
dep_lon = tf.feature_column.numeric_column("DEP_LON")
dep_lon_b = tf.feature_column.bucketized_column(dep_lon, lon_boundaries)
dep_geo_cross = tf.feature_column.crossed_column(['DEP_LAT', 'DEP_LON'], cross_size)
dep_geo_emb = tf.feature_column.embedding_column(dep_geo_cross, 10)

dep_dow = tf.feature_column.categorical_column_with_identity("DEP_DOW", num_buckets=7)
dep_hod = tf.feature_column.categorical_column_with_identity("DEP_T", num_buckets=24)
dep_how = tf.feature_column.crossed_column(["DEP_T", "DEP_DOW"], 7*24)

dep_how_emb = tf.feature_column.embedding_column(dep_how, 10)

#### This describes the path from transformed to input

In [29]:
FEATURE_COLUMNS = [
    dep_how_emb,
    arr_geo_emb,
    dep_geo_emb,
    dep_delay,
    mean_temp_dep,
    mean_temp_arr,
    mean_vis_dep,
    mean_vis_arr,
    wnd_spd_dep,
    wnd_spd_arr,
    diff_lat,
    diff_lon,
    distance
]

#### This is what goes into the network

In [30]:
INPUT_COLUMNS = [
    tf.feature_column.numeric_column('DEP_DELAY'),
    tf.feature_column.numeric_column('MEAN_TEMP_DEP'),
    tf.feature_column.numeric_column('MEAN_VIS_DEP'),
    tf.feature_column.numeric_column('WND_SPD_DEP'),
    tf.feature_column.numeric_column('MEAN_TEMP_ARR'),
    tf.feature_column.numeric_column('MEAN_VIS_ARR'),
    tf.feature_column.numeric_column('WND_SPD_ARR'),
    tf.feature_column.numeric_column('DIFF_LAT'),
    tf.feature_column.numeric_column('DIFF_LON'),
    tf.feature_column.numeric_column('DISTANCE'),
    tf.feature_column.numeric_column("DEP_HOW_EMB", shape=[10]),
    tf.feature_column.numeric_column("ARR_GEO_EMB", shape=[10]),
    tf.feature_column.numeric_column("DEP_GEO_EMB", shape=[10])
]

---
# A custom estimator from our own model

In [31]:
MODEL_DIR="/tmp/model"

In [57]:
def model_fn ( features, labels, mode, params):

    if mode == tf.estimator.ModeKeys.EVAL:
        input_layer = tf.feature_column.input_layer( 
            features, feature_columns=params['feature_columns'])
        out = tf.layers.dense(input_layer, 1, activation=None)
        labels = tf.expand_dims(labels, -1)
        loss = tf.losses.mean_squared_error(labels, out)

        return tf.estimator.EstimatorSpec(
            mode=mode,
            loss = loss,
            #eval_metric_ops={'my_metric': }
        )
    
    elif mode == tf.estimator.ModeKeys.TRAIN:     
        input_layer = tf.feature_column.input_layer( 
            features, feature_columns=params['feature_columns'])
        out = tf.layers.dense(input_layer, 1, activation=None)
        labels = tf.expand_dims(labels, -1)
        loss = tf.losses.mean_squared_error(labels, out)
        optimizer = tf.train.GradientDescentOptimizer(params['learning_rate'])
        train_op = optimizer.minimize(loss, global_step=tf.train.get_or_create_global_step())

        return tf.estimator.EstimatorSpec(  
            mode,
            loss = loss,
            train_op = train_op)
    
    else:
        print(features)
        input_layer = tf.feature_column.input_layer( 
            features, feature_columns=params['feature_columns'])
        print(input_layer)
        out = tf.layers.dense(input_layer, 1, activation=None)
        return tf.estimator.EstimatorSpec(
            mode,
            predictions=out
            )
    

In [58]:
config = tf.estimator.RunConfig(model_dir=MODEL_DIR)

In [59]:
shutil.rmtree(MODEL_DIR, ignore_errors=True)

In [60]:
classifier = tf.estimator.Estimator(
        config=config,
        model_fn=model_fn,
        params={
            'feature_columns': FEATURE_COLUMNS,
            'input_columns': INPUT_COLUMNS,
            'learning_rate': 1e-6
        })

INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3ad2289ed0>, '_model_dir': '/tmp/model', '_protocol': None, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_global_id_in_cluster': 0, '_master': ''}


---
# Training and Serving

In [61]:
METADATA_DIR='gs://going-tfx/prod/metadata/'

In [62]:
def build_serving_input_fn(metadata_dir):
    def _serving_input_fn():
        
      # must be signature stage placeholders
      feature_placeholders = {
        'DEP_HOW_EMB': tf.placeholder(tf.float32, [None]),
        'DEP_GEO_EMB': tf.placeholder(tf.float32, [None]),
        'ARR_GEO_EMB': tf.placeholder(tf.float32, [None]),
        'DEP_DELAY': tf.placeholder(tf.float32, [None]),
        'DIFF_LON': tf.placeholder(tf.float32, [None]),
        'DIFF_LAT': tf.placeholder(tf.float32, [None]),
        'DISTANCE': tf.placeholder(tf.float32, [None]),
        'MEAN_TEMP_DEP': tf.placeholder(tf.float32, [None]),
        'MEAN_TEMP_ARR': tf.placeholder(tf.float32, [None]),
        'MEAN_VIS_DEP': tf.placeholder(tf.float32, [None]),
        'MEAN_VIS_ARR': tf.placeholder(tf.float32, [None]),
        'WND_SPD_DEP': tf.placeholder(tf.float32, [None]),
        'WND_SPD_ARR': tf.placeholder(tf.float32, [None]),
      }

      features = {
          key: tf.expand_dims(tensor, -1)
          for key, tensor in feature_placeholders.items()
      }

      tf_transformer = tft.TFTransformOutput(metadata_dir)
      return tf.estimator.export.ServingInputReceiver(tf_transformer.transform_raw_features(features), placeholders)
    return _serving_input_fn

exporter = tf.estimator.LatestExporter('exporter', build_serving_input_fn(METADATA_DIR))

In [53]:
train_input_fn = make_input_fn(
    a_training_file, mode=tf.estimator.ModeKeys.TRAIN, batch_size=128)

eval_input_fn = make_input_fn(
    an_eval_file, mode=tf.estimator.ModeKeys.EVAL, batch_size=128)  

In [54]:
train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=1000)
eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn, steps = 50, exporters=exporter)

#### At last, start the training!

In [55]:
shutil.rmtree(MODEL_DIR, ignore_errors=True)

In [56]:
tf.estimator.train_and_evaluate(classifier, train_spec=train_spec, eval_spec=eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/model/model.ckpt.
INFO:tensorflow:loss = 1254.9387, step = 1
INFO:tensorflow:global_step/sec: 31.9446
INFO:tensorflow:loss = 2858.9927, step = 101 (3.132 sec)
INFO:tensorflow:global_step/sec: 36.7951
INFO:tensorflow:loss = 2181.9568, step = 201 (2.719 sec)
INFO:tensorflow:global_step/sec: 37.4507
INFO:tensorflow:loss = 2494.0605, step = 301 (2.670 sec)
INFO:tensorflow:global_step/sec: 40.0459
INFO:tensorflow:loss =

ValueError: Unexpected inputs to transform: set(['DISTANCE', 'ARR_GEO_EMB', 'DEP_HOW_EMB', 'DIFF_LAT', 'DEP_GEO_EMB', 'DIFF_LON'])

---
#### Train with the full training set of ~300k records

In [33]:
all_train_files=!ls /tmp/prod/atl_june_train-*-of-00005
all_eval_files=!ls /tmp/prod/atl_june_eval-*-of-00002

In [44]:
full_train_input_fn = make_input_fn(
    all_train_files, mode=tf.estimator.ModeKeys.TRAIN, batch_size=512)

full_eval_input_fn = make_input_fn(
    all_eval_files, mode=tf.estimator.ModeKeys.EVAL, batch_size=512)  

train_spec = tf.estimator.TrainSpec(input_fn=full_train_input_fn, max_steps=2000)
eval_spec = tf.estimator.EvalSpec(input_fn=full_eval_input_fn, steps = 50, exporters=exporter)

In [45]:
tf.estimator.train_and_evaluate(classifier, train_spec=train_spec, eval_spec=eval_spec)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps None or save_checkpoints_secs 600.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/model/model.ckpt.
INFO:tensorflow:loss = 2831.354, step = 1
INFO:tensorflow:global_step/sec: 8.38259
INFO:tensorflow:loss = 1708.9395, step = 101 (11.933 sec)
INFO:tensorflow:global_step/sec: 8.0308
INFO:tensorflow:loss = 1270.7565, step = 201 (12.451 sec)
INFO:tensorflow:global_step/sec: 8.03686
INFO:tensorflow:loss = 1721.4529, step = 301 (12.446 sec)
INFO:tensorflow:global_step/sec: 7.57626
INFO:tensorflow:loss 

({'global_step': 2000, 'loss': 1654.5144},
 ['/tmp/model/export/exporter/1541516558'])